<a href="https://colab.research.google.com/github/HayeonLee/Meta_Learning_Tutorial/blob/master/Meta_Learning_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original Code: https://github.com/abdulfatir/prototypical-networks-tensorflow

# Few-shot Learning 1: Prototypical Network

이 튜토리얼에서 우리는 meta knowledge를 학습하여 few-shot learning을 수행하는 두가지 대표적인 네트워크, [Prototypical Network](https://arxiv.org/abs/1703.05175) (ProtoNet)과 [Model-Agnostic Meta-Learning for Fast Adaptation of Deep Networks](https://arxiv.org/abs/1703.03400) (MAML)을 배울 것입니다.





## Few-shot Learning이란?
(설명 추가 예정)

## Prototypical Network란?
(설명 추가 예정)

## ProtoNet 튜토리얼

이제부터 어떻게 네트워크를 만들고 ProtoNet 학습 시킬지를 배워봅시다.
순서는 다음과 같습니다.
1. Tensorflow와 다른 library들을 불러온다.
2. 데이터셋을 불러온다.
3. 모델을 만든다.
4. loss와 optimizer를 정의한다.
5. Training loop를 정의한다.
6. Training!
7. Test

그럼 하나씩 진행해보도록 합시다.



### Import Tensorflow and other libraries


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
#기존에 설치된 다른 버전의 tensorflow를 제거합니다.
!pip uninstall tensorboard -y
!pip uninstall tensorflow-gpu -y
!pip uninstall tensorflow -y

Uninstalling tensorboard-1.14.0:
  Successfully uninstalled tensorboard-1.14.0
Uninstalling tensorflow-gpu-1.14.0:
  Successfully uninstalled tensorflow-gpu-1.14.0


In [0]:
!pip install tensorflow-gpu==1.14.0 #tensorflow gpu 버전을 설치합니다

  Using cached https://files.pythonhosted.org/packages/76/04/43153bfdfcf6c9a4c38ecdb971ca9a75b9a791bb69a764d652c359aca504/tensorflow_gpu-1.14.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/91/2d/2ed263449a078cd9c8a9ba50ebd50123adf1f8cfbea1492f9084169b89d9/tensorboard-1.14.0-py3-none-any.whl


In [0]:
import tensorflow as tf # tensorflow를 import해줍니다

In [0]:
tf.__version__ # 내가 사용할 tensorflow의 버전을 나타냅니다

'1.14.0'

In [0]:
if tf.test.gpu_device_name():
  print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
  print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [0]:
# 필요한 라이브러리를 import합니다.
%matplotlib inline
from PIL import Image
import numpy as np
import tensorflow as tf
import os
import glob
import matplotlib.pyplot as plt

In [0]:
!mkdir -p data/omniglot/data
!mkdir -p data/omniglot/split
!wget -O data/omniglot/split/trainval.txt https://github.com/kvpratama/prototypical-networks-tensorflow/blob/master/data/omniglot/splits/trainval.txt?raw=true
!wget -O data/omniglot/split/train.txt https://github.com/kvpratama/prototypical-networks-tensorflow/blob/master/data/omniglot/splits/train.txt?raw=true
!wget -O data/omniglot/split/test.txt https://github.com/kvpratama/prototypical-networks-tensorflow/blob/master/data/omniglot/splits/test.txt?raw=true
!wget -O images_background.zip https://github.com/brendenlake/omniglot/blob/master/python/images_background.zip?raw=true
!wget -O images_evaluation.zip https://github.com/brendenlake/omniglot/blob/master/python/images_evaluation.zip?raw=true
!unzip images_background.zip -d data/omniglot/data
!unzip images_evaluation.zip -d data/omniglot/data
!mv data/omniglot/data/images_background/* data/omniglot/data/
!mv data/omniglot/data/images_evaluation/* data/omniglot/data/
!rmdir data/images_background
!rmdir data/images_evaluation

In [0]:
!ls data/omniglot

data  split


In [0]:
def conv_block(inputs, out_channels, name='conv'):
    with tf.variable_scope(name):
        conv = tf.layers.conv2d(inputs, out_channels, kernel_size=3, padding='SAME')
        conv = tf.contrib.layers.batch_norm(conv, updates_collections=None, decay=0.99, scale=True, center=True)
        conv = tf.nn.relu(conv)
        conv = tf.contrib.layers.max_pool2d(conv, 2)
        return conv

In [0]:
def encoder(x, h_dim, z_dim, reuse=False):
    with tf.variable_scope('encoder', reuse=reuse):
        net = conv_block(x, h_dim, name='conv_1')
        net = conv_block(net, h_dim, name='conv_2')
        net = conv_block(net, h_dim, name='conv_3')
        net = conv_block(net, z_dim, name='conv_4')
        net = tf.contrib.layers.flatten(net)
        return net

In [0]:
def euclidean_distance(a, b):
    # a.shape = N x D
    # b.shape = M x D
    N, D = tf.shape(a)[0], tf.shape(a)[1]
    M = tf.shape(b)[0]
    a = tf.tile(tf.expand_dims(a, axis=1), (1, M, 1))
    b = tf.tile(tf.expand_dims(b, axis=0), (N, 1, 1))
    return tf.reduce_mean(tf.square(a - b), axis=2)

In [0]:
n_epochs = 20
n_episodes = 100
n_way = 60
n_shot = 5
n_query = 5
n_examples = 20
im_width, im_height, channels = 28, 28, 1
h_dim = 64
z_dim = 64

In [0]:
# Load Train Dataset
root_dir = './data/omniglot'
train_split_path = os.path.join(root_dir, 'split', 'train.txt')
with open(train_split_path, 'r') as train_split:
    train_classes = [line.rstrip() for line in train_split.readlines()]
n_classes = len(train_classes)
train_dataset = np.zeros([n_classes, n_examples, im_height, im_width], dtype=np.float32)
for i, tc in enumerate(train_classes):
    alphabet, character, rotation = tc.split('/')
    rotation = float(rotation[3:])
    im_dir = os.path.join(root_dir, 'data', alphabet, character)
    im_files = sorted(glob.glob(os.path.join(im_dir, '*.png')))
    for j, im_file in enumerate(im_files):
        im = 1. - np.array(Image.open(im_file).rotate(rotation).resize((im_width, im_height)), np.float32, copy=False)
        train_dataset[i, j] = im
print(train_dataset.shape)

(4112, 20, 28, 28)


In [0]:
x = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
q = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
x_shape = tf.shape(x)
q_shape = tf.shape(q)
num_classes, num_support = x_shape[0], x_shape[1]
num_queries = q_shape[1]
y = tf.placeholder(tf.int64, [None, None])
y_one_hot = tf.one_hot(y, depth=num_classes)
emb_x = encoder(tf.reshape(x, [num_classes * num_support, im_height, im_width, channels]), h_dim, z_dim)
emb_dim = tf.shape(emb_x)[-1]
emb_x = tf.reduce_mean(tf.reshape(emb_x, [num_classes, num_support, emb_dim]), axis=1)
emb_q = encoder(tf.reshape(q, [num_classes * num_queries, im_height, im_width, channels]), h_dim, z_dim, reuse=True)
dists = euclidean_distance(emb_q, emb_x)
log_p_y = tf.reshape(tf.nn.log_softmax(-dists), [num_classes, num_queries, -1])
ce_loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_one_hot, log_p_y), axis=-1), [-1]))
acc = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(log_p_y, axis=-1), y)))

W0707 12:29:44.478682 139880249501568 deprecation.py:323] From <ipython-input-4-484dc49fabd8>:3: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0707 12:29:44.486674 139880249501568 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0707 12:29:47.129935 139880249501568 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for 

In [0]:
train_op = tf.train.AdamOptimizer().minimize(ce_loss)

In [0]:
sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [0]:
for ep in range(n_epochs):
    for epi in range(n_episodes):
        epi_classes = np.random.permutation(n_classes)[:n_way]
        support = np.zeros([n_way, n_shot, im_height, im_width], dtype=np.float32)
        query = np.zeros([n_way, n_query, im_height, im_width], dtype=np.float32)
        for i, epi_cls in enumerate(epi_classes):
            selected = np.random.permutation(n_examples)[:n_shot + n_query]
            support[i] = train_dataset[epi_cls, selected[:n_shot]]
            query[i] = train_dataset[epi_cls, selected[n_shot:]]
        support = np.expand_dims(support, axis=-1)
        query = np.expand_dims(query, axis=-1)
        labels = np.tile(np.arange(n_way)[:, np.newaxis], (1, n_query)).astype(np.uint8)
        _, ls, ac = sess.run([train_op, ce_loss, acc], feed_dict={x: support, q: query, y:labels})
        if (epi+1) % 50 == 0:
            print('[epoch {}/{}, episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(ep+1, n_epochs, epi+1, n_episodes, ls, ac))

[epoch 1/20, episode 50/100] => loss: 2.09691, acc: 0.68000
[epoch 1/20, episode 100/100] => loss: 1.40069, acc: 0.81667
[epoch 2/20, episode 50/100] => loss: 1.10882, acc: 0.85667
[epoch 2/20, episode 100/100] => loss: 0.94011, acc: 0.84333
[epoch 3/20, episode 50/100] => loss: 0.63034, acc: 0.91333
[epoch 3/20, episode 100/100] => loss: 0.49404, acc: 0.93000
[epoch 4/20, episode 50/100] => loss: 0.42604, acc: 0.92667
[epoch 4/20, episode 100/100] => loss: 0.35078, acc: 0.96333
[epoch 5/20, episode 50/100] => loss: 0.34698, acc: 0.95000
[epoch 5/20, episode 100/100] => loss: 0.27430, acc: 0.97000
[epoch 6/20, episode 50/100] => loss: 0.26539, acc: 0.95667
[epoch 6/20, episode 100/100] => loss: 0.20937, acc: 0.96667
[epoch 7/20, episode 50/100] => loss: 0.17970, acc: 0.97333
[epoch 7/20, episode 100/100] => loss: 0.17806, acc: 0.98000
[epoch 8/20, episode 50/100] => loss: 0.14741, acc: 0.98333
[epoch 8/20, episode 100/100] => loss: 0.19914, acc: 0.96667
[epoch 9/20, episode 50/100] => 